In [1]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # or any {'0', '1', '2'}
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import tensorflow
from keras.utils import to_categorical
# import train_test_split
from sklearn.model_selection import train_test_split

# import 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Read dataset:
df = pd.read_csv('../Data/dataset/VETUSDT_labelled.csv')
df.drop(['original_close', 'original_open', 'original_high', 'original_low', 'original_volume', 'event_time'], axis=1, inplace=True)

df['balance'] = 100
df['coin_amount'] = 0
X = df.drop(['action'], axis=1)
y = df['action']
y = to_categorical(y, num_classes=3)


In [ ]:
learning_rate = 0.00002
input_shape = (100, 19)

sequence_length = 100

In [ ]:
# create sequences as generator
def create_sequences(df, sequence_length):
    sequences = []
    for x in range(sequence_length, len(df) - sequence_length):
        sequences.append(df[x - sequence_length:x]) 
    return np.array(sequences)


In [ ]:
def create_model():
    model = tf.keras.Sequential()

    optimizer = Adam(learning_rate=learning_rate)
    model.add(layers.LSTM(200, input_shape=input_shape, return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(32))
    model.add(layers.Dense(16, activation='linear'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['mae', 'accuracy'])
    return model

In [ ]:
X_train = create_sequences(X, sequence_length)
y_train = create_sequences(y, sequence_length)

model = create_model()

model.fit(X_train, y_train, batch_size=64, epochs=1000, verbose=1)




Epoch 1/1000


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
model.save('pretrained_lstm_layers.h5')

In [ ]:
base_model = tf.keras.models.load_model('pretrained_lstm_layers.h5')

# remove last layer
new_model = tf.keras.models.Sequential(base_model.layers[:-1])


new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100, 200)          176000    
                                                                 
 dropout_3 (Dropout)         (None, 100, 200)          0         
                                                                 
 lstm_5 (LSTM)               (None, 100, 128)          168448    
                                                                 
 dropout_4 (Dropout)         (None, 100, 128)          0         
                                                                 
 lstm_6 (LSTM)               (None, 100, 64)           49408     
                                                                 
 dropout_5 (Dropout)         (None, 100, 64)           0         
                                                                 
 lstm_7 (LSTM)               (None, 32)               